In [11]:
import pandas as pd
import numpy as np
from datetime import timedelta

import os
import pickle
from config import BASE_DIR
print(os.getcwd())
os.chdir(BASE_DIR)
import scripts.functions as fn 
from scripts.indicators import psar,zigzag




C:\Users\lbisa\Mi unidad (leonardo.bisaro@gmail.com)\Cripto\python\trd_app


In [12]:
klines_file  = './backtest/klines/1h04/Completo_BTCUSDT_1h04_2021-01-01_2023-07-31.DataFrame'
klines_file  = './backtest/klines/1h01/Completo_BTCUSDT_1h01_2021-01-01_2023-07-31.DataFrame'
klines_file = './backtest/klines/2d01/Lateral_BTCUSDT_2d01_2023-06-20_2023-10-09.DataFrame'
klines_file  = './backtest/klines/2d01/Full_BTCUSDT_2d01_2019-01-01_2024-06-20.DataFrame'
klines_file  = './backtest/klines/1h01/Full_BTCUSDT_1h01_2019-01-01_2024-06-20.DataFrame'

klines_file  = './backtest/klines/1h01/Completo_BTCUSDT_1h01_2021-01-01_2023-07-31.DataFrame'
klines_file  = './backtest/klines/0m15/Completo_BTCUSDT_0m15_2021-01-01_2023-07-31.DataFrame'
klines_file  = './backtest/klines/2d01/Completo_BTCUSDT_2d01_2021-01-01_2023-07-31.DataFrame'

with open(klines_file, 'rb') as file:
    df = pickle.load(file)
    #data.set_index('datetime',inplace=True)

#df = fn.ohlc_mirror_v(df)

#Filtrar rango de fechas
#ini = pd.to_datetime('2018-06-26')
#fin = pd.to_datetime('2028-05-01')
#df = df[(df['datetime']>=ini)&(df['datetime']<=fin)]
#df.reset_index(inplace=True)
#df.drop(columns=['index'],inplace=True)


#df.drop(columns=['volume'],inplace=True)

#Quitar parte del dataframe
#df = df[500:1500]
#df.reset_index(inplace=True)
#df.drop(columns=['index'],inplace=True)


In [ ]:
#df['price'] = (df['high']+df['low']+df['open']+df['close']*2)/5
df = psar(df)
df['pivots'] = np.where((df['psar_high']>0) & (df['psar_low'].shift(1)>0), df['psar_high'] , None)
df['pivots'] = np.where((df['psar_low']>0) & (df['psar_high'].shift(1)>0), df['psar_low'] , df['pivots'])

pivots = df['pivots'][df['pivots']>0]
df['pivots1'] = pivots.shift(1)
df['pivots2'] = pivots.shift(2)

df['buy']  = np.where((df['pivots'] > df['pivots2']) & (df['pivots'] < df['pivots1']), df['close'], None)
df['sell'] = np.where((df['pivots'] < df['pivots2']) & (df['pivots'] < df['pivots1']), df['close'], None)

df['tp'] = np.where((df['psar_high']>0) & (df['psar_low'].shift(1)>0), df['psar_high'] , None)
df['tp'].ffill(inplace=True)
df['sl'] = np.where((df['psar_low']>0) & (df['psar_high'].shift(1)>0), df['psar_low'] , None)
df['sl'].ffill(inplace=True)

In [16]:
indicators = [
     {'col': 'sl','name': 'sl','color': 'red','row': 1, 'mode':'lines',},
     {'col': 'tp','name': 'tp','color': 'green','row': 1, 'mode':'lines',},
     {'col': 'pivots','name': 'Pivots','color': 'yellow','row': 1, 'mode':'lines',},
     #{'col': 'st_trigger','name': 'ST Trigger','color': 'red','row': 1, 'mode':'markers',},
     #{'col': 'st_high','name': 'ST High','color': 'green','row': 1, 'mode':'markers',},
        ]

indicators_out = [
     #{'col': 'st_trend','name': 'ST','color': 'white','row': 2, 'mode':'lines',},
        ]

#https://plotly.com/python/reference/scatter/#scatter-marker-symbol
#https://plotly.com/python/reference/scatter/#scatter-marker-symbol
events = [
        {'df':df,'col':'buy','name': 'BUY','color': 'green','symbol': 'circle-dot' }, 
        {'df':df,'col':'sell','name': 'SELL','color': 'red','symbol': 'circle-dot' }, 
     ]
if 'volume' in df.columns:
        df.drop(columns=['volume'],inplace=True)

fig = fn.ohlc_chart(df,indicators=indicators,indicators_out=indicators_out,events=events) #
fig.show()